In [ ]:
import numpy as np
import scipy.special as sc
import matplotlib.pyplot as plt
from scipy.io import loadmat
from ipywidgets import interact, interactive, widgets

from scipy.constants import h, c, k, e, m_e, epsilon_0,physical_constants

k_ev = physical_constants['Boltzmann constant in eV/K'][0]
sigma_sb = physical_constants['Stefan-Boltzmann constant'][0]
L = 2.44e-8

## Definitions

In [ ]:
def rd_current(phi, T):
    """
    Thermionic emission current density in A/m**2
    """
    A = 4 * np.pi * m_e * k**2 * e / h**3
    
    return A * T**2 * np.exp(-phi / (k_ev * T))

def Vretard(J,phi_c, T):
    """
    Thermionic emission current density in A/m**2
    """
    A = 4 * np.pi * m_e * k**2 * e / h**3
    
    return -np.log(J/(A * T**2)) *  (k_ev * T)-phi_c


def x0_fun(J,T):
    x0 = (epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/J**(1/2)
    return x0

def xiFromJ(J,x,xm,T):
    x0 = (epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/J**(1/2)
    xi = (x-xm)/x0
    
    return xi

def JFromxi(xi,x,xm,T):
    x0 = (x-xm)/xi
    J = ((epsilon_0**2 * k**3/2/np.pi/m_e/e**2)**(1/4)*T**(3/4)/x0)**2
    
    return J

def calculate_currfraction(T, V_gap, drefprob, E_arr=None):
    if E_arr is None:
        if V_gap > 0:
            if (1 - drefprob) * e * V_gap / k / T < 700:
                frac= (1-drefprob) * (1 + drefprob * e * V_gap / k / T + drefprob * (1 - drefprob) * (e * V_gap / k / T) ** 2 * np.exp((1 - drefprob) * e * V_gap / k / T ) * sc.expi(-(1 - drefprob) * e * V_gap / k / T))
            else:
                t = np.linspace(0,1e2,1000000)
                frac= (1-drefprob) * (1 + drefprob * e * V_gap / k / T + drefprob * (1 - drefprob) * (e * V_gap / k / T) ** 2 * np.trapz(-np.exp(-t)/(t+(1 - drefprob) * e * V_gap / k / T), t))
        else:
            frac = (1-drefprob)
    else:
        if len(drefprob) != len(E_arr):
            raise RuntimeError("energy array (E_arr) and diffusive reflectivity (drefprob) must be in the same size")
        frac = np.trapz(np.exp(-E_arr / k / T) * (1 - drefprob) / (1 - drefprob * e * V_gap / (e * V_gap + E_arr)) * E_arr / (k * T) ** 2, E_arr)
    
    return frac

In [ ]:
def calculate_J(phi_e, phi_c, T_e, d, gammas, xim, xip, l_sat = False):
    J_sat = rd_current(phi_e,T_e)
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e = np.interp(xi_e,xip,gammas)
    V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
#     print("gamma_e is:", gamma_e)
#     print("V_sat is:",V_sat)
    
    if V_sat > phi_e-phi_c or l_sat:
        J_res = J_sat
        print("No reflection. T_e:",round(T_e - 273.15),"C, saturation current: ", J_res)
        return J_res
    
    # first guess of the critical point:
    J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
    gamma_E = np.log(J_sat/J_c_0)
    V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
    gamma_e_arr = np.log(J_sat/J_arr)
    excludeind = np.where(gamma_e_arr<0)
    np.put(gamma_e_arr,excludeind,0)
    xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
    xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
    np.put(xi_c_arr,excludeind,-1)
    xi_c = min(xi_c_arr[xi_c_arr>=0])
    J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
    V_r = Vretard(J_r, phi_c, T_e)
#     print("V_cr", V_r)
    
    if V_r < phi_e-phi_c:
        J_res = rd_current(phi_c + phi_e - phi_c,T_e)
        print("No reflection. T_e:",round(T_e - 273.15),"C, retard current: ", J_res)
        return J_res
    
    # space charge limited iv:
    n_scl = 1000000
    J_scl = np.linspace(J_sat,J_r,n_scl)
    gamma_e_scl = np.log(J_sat/J_scl)
    xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
    xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
    gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
    V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k*T_e/e
    J_res = np.interp(phi_e-phi_c, V_scl, J_scl)
#     print("No reflection. T_e:",round(T_e - 273.15),"C, SCL current: ", J_res)
    return J_res

def calculate_J_sr(phi_e, phi_c, T_e, d, gammas, xim, xip, srefprob, l_sat = False):
    J_sat = rd_current(phi_e,T_e)
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e = np.interp(xi_e,xip,gammas)
    V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
#     print("V_sat is:",V_sat)
    
    if V_sat > phi_e-phi_c or l_sat:
        J_res = J_sat * (1 - srefprob)
        print("Specular reflection. T_e:",round(T_e - 273.15),"C, saturation current: ", J_res)
        return J_res
    
    # first guess of the critical point:
    J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
    gamma_E = np.log(J_sat/J_c_0)
    V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
    gamma_e_arr = np.log(J_sat/J_arr)
    excludeind = np.where(gamma_e_arr<0)
    np.put(gamma_e_arr,excludeind,0)
    xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
    xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
    np.put(xi_c_arr,excludeind,-1)
    xi_c = min(xi_c_arr[xi_c_arr>=0])
    J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
    V_r = Vretard(J_r, phi_c, T_e)
#     print("V_cr", V_r)
    
    if V_r < phi_e-phi_c:
        J_res = rd_current(phi_c + phi_e - phi_c,T_e) * (1 - srefprob)
        print("Specular reflection. T_e:",round(T_e - 273.15),"C, retard current: ", J_res)
        return J_res
    
    # space charge limited iv:
    n_scl = 1000000
    J_scl = np.linspace(J_sat,J_r,n_scl)
    gamma_e_scl = np.log(J_sat/J_scl)
    xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
    xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
    gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
    V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k*T_e/e
    J_res = np.interp(phi_e-phi_c, V_scl, J_scl) * (1 - srefprob)
    print("Specular reflection. T_e:",round(T_e - 273.15),"C, SCL current: ", J_res)
    return J_res

def calculate_J_dr(phi_e, phi_c, T_e, d, gammas, xim, xip, drefprob, l_sat = False):
    J_sat = rd_current(phi_e,T_e)
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e = np.interp(xi_e,xip,gammas)
    V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
#     print("V_sat is:",V_sat)
    if V_sat > phi_e-phi_c or l_sat:
        J_res = J_sat * calculate_currfraction(T_e, phi_e-phi_c, drefprob)
        print("Diffuse reflection. T_e:",round(T_e - 273.15),"C, saturation current: ", J_res)
        return J_res
    
    # first guess of the critical point:
    J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
    gamma_E = np.log(J_sat/J_c_0)
    V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
    gamma_e_arr = np.log(J_sat/J_arr)
    excludeind = np.where(gamma_e_arr<0)
    np.put(gamma_e_arr,excludeind,0)
    xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
    xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
    np.put(xi_c_arr,excludeind,-1)
    xi_c = min(xi_c_arr[xi_c_arr>=0])
    J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
    V_r = Vretard(J_r, phi_c, T_e)
#     print("V_cr", V_r)
    
    if V_r < phi_e-phi_c:
        J_res = rd_current(phi_c + phi_e - phi_c,T_e) * calculate_currfraction(T_e, phi_e - phi_c, drefprob)
        print("Diffuse reflection. T_e:",round(T_e - 273.15),"C, retard current: ", J_res)
        return J_res
    
    # space charge limited iv:
    n_scl = 1000000
    J_scl = np.linspace(J_sat,J_r,n_scl)
    gamma_e_scl = np.log(J_sat/J_scl)
    xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
    xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
    gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
    V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k_ev*T_e
    J_scl0 = np.interp(phi_e-phi_c, V_scl, J_scl)
    V_a0 = (np.log(J_sat/J_scl0))*k_ev*T_e
    J_res = J_scl0 * calculate_currfraction(T_e, V_a0, drefprob)
    print("Diffuse reflection. T_e:",round(T_e - 273.15),"C, SCL current: ", J_res)
    return J_res

def calculate_efficiency(A_em,J_em, J_coll, phi_em, T_em, rho_em,
               rho_ew, rho_cw, phi_coll, T_coll, phi_m,
               emiss_eff, T_env=293.15,applied_voltage=0.0,
               rho_load=0, single_cycle=True):
    """
    Calculate the TEC efficieny.
    All power terms should be calculated to give W/cm**2
    Based on S. Meir et al. J. Renewable Sustainable Energy 2013.
    Args:
        rho_ew: Effective emitter wiring resistivity (Ohms*cm)
        J_em: Emitter current density (A/cm**2)
        J_coll: Collector current density (A/cm**2)
        phi_em: Emitter work function (eV)
        T_em: Emitter temperature (K)
        rho_cw: Effective collector wiring resistivity (Ohms*cm)
        phi_coll: Collector work function (eV)
        T_coll: Collector temperature (K)
        emiss_eff: Emissivity (none)
        T_env: Ambient temperature (K)
        rho_load: Effective load resistivity (Ohms*cm)
                  if set < 0 then perfect matching is assumed -> V_load = phi_em - phi_col.
        single_cycle: True if single cycle, False if combined cycle
        **kwargs: Catch unused arguments

    Returns:
        Efficiency (none)
    """
    
    # Turning off analytic backward current. Is a small effect but creates misleading results in low forward current
    #  cases encountered frequently in optimization

    # Modify measured J_ec (emitter to collector current) to remove analytical collector produced current
    J_load = (J_em - J_coll)

    # P_ew
    P_ew = 0.5 * (L / (rho_em * A_em) * (T_em - T_env) ** 2 - (J_load) ** 2 * A_em * rho_ew)

    # P_cw
    P_cw = -0.5 * (L / (rho_cw * A_em) * (T_coll - T_env) ** 2 - (J_load) ** 2 * A_em * rho_cw)

    # P_r
    P_r = emiss_eff * sigma_sb * (T_em ** 4 - T_coll ** 4) * 1e-4

    # P_ec (electron cooling power)
    P_ec = J_em * (phi_em + phi_m + 2 * k_ev * T_em) - J_coll * (phi_em + phi_m + 2 * k_ev * T_coll)

    
    # P_eh (electron heating of the collector)
    P_eh = J_em * (phi_coll + 2 * k_ev * T_em) - J_coll * (phi_coll + 2 * k_ev * T_coll)

    # P_load
    V_lead = J_load * A_em * (rho_cw + rho_ew)
    # TODO: Check R_total for adjustment in rho
    if rho_load > 0:
        R_total = rho_load
        V_load = R_total * A_em * J_load - V_lead
    else:
        V_load = applied_voltage - V_lead
    if V_load < 0: V_load = 0
    if J_load < 0: J_load = 0  
    P_load = J_load * V_load
    

    if single_cycle:

        eta = (P_load) / (P_ec + P_r + P_ew )
        efficiency_data = {}
        efficiency_data['P_ew'] = P_ew
        efficiency_data['P_r'] = P_r
        efficiency_data['P_ec'] = P_ec
        efficiency_data['P_load'] = P_load
        efficiency_data['eta'] = eta
    else:
        eta_s = 0.32
        Q_out = P_eh + P_r + P_cw
        eta = (P_load + eta_s * Q_out) / (P_ec + P_r + P_ew )

        efficiency_data = {}
        efficiency_data['P_ew'] = P_ew
        efficiency_data['P_r'] = P_r
        efficiency_data['P_ec'] = P_ec
        efficiency_data['P_load'] = P_load
        efficiency_data['P_s'] = eta_s * Q_out
        efficiency_data['eta'] = eta
        

    return efficiency_data

## _Start i-v calculations_

In [ ]:
srefprob = 0.5
drefprob = 0.5
srefprob_str = str(srefprob).replace('.','')
drefprob_str = str(drefprob).replace('.','')
#Load gamma-xi table calculated via Nintegrate in Mathematica (with wider range of gamma) or via trapz in Python
data_source = 'mathematica' #'python'#
if data_source == 'mathematica':
    data = np.genfromtxt('./data_table.csv',delimiter=',')#_0to100highres
    gammas = data[:,0]
    xim = np.real(data[:,1])
    xip = np.real(data[:,2])
    data = np.genfromtxt('./data_table_sr'+str(drefprob)+'.csv',delimiter=',')#
    gammas_dr = data[:,0]
    xim_dr = np.real(data[:,1])
    xip_dr = np.real(data[:,2])
    data = np.genfromtxt('./data_table_sr'+str(srefprob)+'.csv',delimiter=',')
    gammas_sr = data[:,0]
    xim_sr = np.real(data[:,1])
    xip_sr = np.real(data[:,2])
elif data_source == 'python':
    gammas,xim,xip = np.load('gamma_xi_table_v2.npy')#
    gammas_sr,xim_sr,xip_sr = np.load('gamma_xi_table_sr'+str(srefprob)+'_v2.npy')#
    gammas_dr,xim_dr,xip_dr = np.load('gamma_xi_table_dr'+str(drefprob)+'_v2.npy')#

In [ ]:
T_es = np.concatenate((np.linspace(1000,2000,201), np.linspace(3000,5000,3)))#C

phi_e = 2.174# 2.3#2.22#eV
phi_c = 0.381#1.73#2.1 #eV
dgap = '1'
d = float(dgap)*1e-6 #m 3.16


if_sat = False
if_Child = False
if_backemission = True
A_em = 1
T_coll = 50 + 273.15
emiss_eff = 0.2
T_env = 293.15
rho_em = 1e-3
rho_cw = 1e-3
rho_ew = 1e-3


In [ ]:
data1_no = []
data2_no = []
data3_no = []
data1_sr = []
data2_sr = []
data3_sr = []
data1_dr = []
data2_dr = []
data3_dr = []

if if_backemission:
    J_coll = rd_current(phi_c, T_coll)*1e-4
else:
    J_coll = 0
    
for T_ei in T_es:
    strT = str(int(T_ei))
    T_e = T_ei + 273.15  #K
    
    # No reflection
    # saturation point:
    J_sat = rd_current(phi_e,T_e)#sources.j_rd(T_e, phi_e)#
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e = np.interp(xi_e,xip,gammas)
    V_sat = phi_e-phi_c-gamma_e*k_ev*T_e
#     print("V_sat:",V_sat)

    # first guess of the critical point:
    J_c_0 = JFromxi(max(np.abs(xim)),0,d,T_e)
    gamma_E = np.log(J_sat/J_c_0)
    V_c_0 = -(gamma_E*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr = np.exp(np.linspace(-6,3,100000))*J_c_0
    gamma_e_arr = np.log(J_sat/J_arr)
    excludeind = np.where(gamma_e_arr<0)
    np.put(gamma_e_arr,excludeind,0)
    xi_e_arr = np.interp(gamma_e_arr, gammas,xim)
    xi_c_arr = d/x0_fun(J_arr, T_e) + xi_e_arr
    np.put(xi_c_arr,excludeind,-1)
    xi_c = min(xi_c_arr[xi_c_arr>=0])
    J_r = J_arr[np.where(xi_c_arr==xi_c)[0][0]]
    V_r = Vretard(J_r, phi_c, T_e)
#     print("V_cr:", V_r)

    # space charge limited iv:
    n_scl = 100000#00
    J_scl = np.linspace(J_sat,J_r,n_scl)
    gamma_e_scl = np.log(J_sat/J_scl)
    xi_e_scl = np.interp(gamma_e_scl, gammas,xim)
    xi_c_scl = d/x0_fun(J_scl, T_e) + xi_e_scl
    gamma_c_scl = np.interp(xi_c_scl,xip,gammas)
    V_scl = phi_e-phi_c+(gamma_e_scl-gamma_c_scl)*k*T_e/e

    # construct the entire i-v table:
    n_sat = 100
    n_r = 100

    J_sat_arr = np.full(n_sat, J_sat)
    V_sat_arr = np.linspace(V_sat-4,V_sat,n_sat)
    V_r_arr = np.linspace(V_r,V_r+ 4, n_r)
    J_r_arr = rd_current(phi_c+V_r_arr, T_e)

    V_data = np.concatenate((V_sat_arr,V_scl,V_r_arr))#
    J_data = np.concatenate((J_sat_arr,J_scl,J_r_arr))#
    efficiencies = np.zeros_like(V_data)
    powers = np.zeros_like(V_data)
    for i,V in enumerate(V_data):
        temp = calculate_efficiency(A_em, J_data[i]*1e-4, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll, np.log(J_sat/J_data[i])*k*T_e/e,
                   emiss_eff, T_env, applied_voltage=V_data[i], single_cycle=1)
        efficiencies[i] = temp['eta']
        powers[i] = temp['P_load']
    ind1 = n_sat
    ind2 = np.argmax(powers)
    ind3 = np.argmax(efficiencies)
    data1_no.append([T_ei, V_data[ind1], J_data[ind1], powers[ind1], efficiencies[ind1]])
    data2_no.append([T_ei, V_data[ind2], J_data[ind2], powers[ind2], efficiencies[ind2]])
    data3_no.append([T_ei, V_data[ind3], J_data[ind3], powers[ind3], efficiencies[ind3]])
    
    
    # Specular reflection
    # saturation point:
    J_sat = rd_current(phi_e,T_e)#sources.j_rd(T_e, phi_e)#
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e_sr = np.interp(xi_e,xip_sr,gammas_sr)
    # gamma_e = 10
    V_sat_sr = phi_e-phi_c-gamma_e_sr*k_ev*T_e
#     print("V_sat_sr is:",V_sat_sr)

    # first guess of the critical point:
    J_c_0_sr = JFromxi(max(np.abs(xim_sr)),0,d,T_e)
    gamma_E_sr = np.log(J_sat/J_c_0_sr)
    V_c_0_sr = -(gamma_E_sr*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr_sr = np.exp(np.linspace(-6,3,10000))*J_c_0_sr
    gamma_e_arr_sr = np.log(J_sat/J_arr_sr)
    excludeind = np.where(gamma_e_arr_sr<0)
    np.put(gamma_e_arr_sr,excludeind,0)
    xi_e_arr_sr = np.interp(gamma_e_arr_sr, gammas_sr,xim_sr)
    xi_c_arr_sr = d/x0_fun(J_arr_sr, T_e) + xi_e_arr_sr
    np.put(xi_c_arr_sr,excludeind,-1)
    xi_c_sr = min(xi_c_arr_sr[xi_c_arr_sr>=0])
    J_r_sr = J_arr_sr[np.where(xi_c_arr_sr==xi_c_sr)[0][0]]
    V_r_sr = Vretard(J_r_sr, phi_c, T_e)
#     print("V_cr_sr", V_r_sr)

    # space charge limited iv:

    J_scl_sr = np.linspace(J_sat,J_r_sr,n_scl)
    gamma_e_scl_sr = np.log(J_sat/J_scl_sr)
    xi_e_scl_sr = np.interp(gamma_e_scl_sr, gammas_sr,xim_sr)
    xi_c_scl_sr = d/x0_fun(J_scl_sr, T_e) + xi_e_scl_sr
    gamma_c_scl_sr = np.interp(xi_c_scl_sr,xip_sr,gammas_sr)
    V_scl_sr = phi_e-phi_c+(gamma_e_scl_sr-gamma_c_scl_sr)*k*T_e/e

    # construct the entire i-v table:
    J_sat_arr_sr = np.full(n_sat, J_sat)
    V_sat_arr_sr = np.linspace(V_sat_sr-4,V_sat_sr,n_sat)
    V_r_arr_sr = np.linspace(V_r_sr,V_r_sr+ 4, n_r)
    J_r_arr_sr = rd_current(phi_c+V_r_arr_sr, T_e)

    V_data_sr = np.concatenate((V_sat_arr_sr,V_scl_sr,V_r_arr_sr))#
    J_data_sr = np.concatenate((J_sat_arr_sr,J_scl_sr,J_r_arr_sr)) * (1 - srefprob)#
    efficiencies_sr = np.zeros_like(V_data_sr)
    powers_sr = np.zeros_like(V_data_sr)
    for i,V in enumerate(V_data_sr):
        temp = calculate_efficiency(A_em, J_data_sr[i]*1e-4, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll, np.log(J_sat/J_data_sr[i]*(1 - srefprob))*k*T_e/e,
                   emiss_eff, T_env, applied_voltage=V_data_sr[i], single_cycle=1)
        efficiencies_sr[i] = temp['eta']
        powers_sr[i] = temp['P_load']
    ind1 = n_sat
    ind2 = np.argmax(powers_sr)
    ind3 = np.argmax(efficiencies_sr)
    data1_sr.append([T_ei, V_data_sr[ind1], J_data_sr[ind1], powers_sr[ind1], efficiencies_sr[ind1]])
    data2_sr.append([T_ei, V_data_sr[ind2], J_data_sr[ind2], powers_sr[ind2], efficiencies_sr[ind2]])
    data3_sr.append([T_ei, V_data_sr[ind3], J_data_sr[ind3], powers_sr[ind3], efficiencies_sr[ind3]])
    
    # Diffuse reflection
    # saturation point:
    J_sat = rd_current(phi_e,T_e)#sources.j_rd(T_e, phi_e)#
    xi_e = xiFromJ(J_sat,d,0,T_e)
    gamma_e_dr = np.interp(xi_e,xip_dr,gammas_dr)
    V_sat_dr = phi_e-phi_c-gamma_e_dr*k_ev*T_e
#     print("V_sat_dr is:",V_sat_dr)

    # first guess of the critical point:
    J_c_0_dr = JFromxi(max(np.abs(xim_dr)),0,d,T_e)
    gamma_E_dr = np.log(J_sat/J_c_0_dr)
    V_c_0_dr = -(gamma_E_dr*k*T_e/e-phi_e+phi_c)

    # critical point:
    J_arr_dr = np.exp(np.linspace(-6,3,10000))*J_c_0_dr
    gamma_e_arr_dr = np.log(J_sat/J_arr_dr)
    excludeind = np.where(gamma_e_arr_dr<0)
    np.put(gamma_e_arr_dr,excludeind,0)
    xi_e_arr_dr = np.interp(gamma_e_arr_dr, gammas_dr,xim_dr)
    xi_c_arr_dr = d/x0_fun(J_arr_dr, T_e) + xi_e_arr_dr
    np.put(xi_c_arr_dr,excludeind,-1)
    xi_c_dr = min(xi_c_arr_dr[xi_c_arr_dr>=0])
    J_r_dr = J_arr_dr[np.where(xi_c_arr_dr==xi_c_dr)[0][0]]
    V_r_dr = Vretard(J_r_dr, phi_c, T_e)
#     print("V_cr_dr", V_r_dr)

    # space charge limited iv:
    J_scl_dr = np.linspace(J_sat,J_r_dr,n_scl)
    gamma_e_scl_dr = np.log(J_sat/J_scl_dr)
    xi_e_scl_dr = np.interp(gamma_e_scl_dr, gammas_dr,xim_dr)
    xi_c_scl_dr = d/x0_fun(J_scl_dr, T_e) + xi_e_scl_dr
    gamma_c_scl_dr = np.interp(xi_c_scl_dr,xip_dr,gammas_dr)
    V_scl_dr = phi_e-phi_c+(gamma_e_scl_dr-gamma_c_scl_dr)*k*T_e/e

    # construct the entire i-v table:
    J_sat_arr_dr = np.full(n_sat, J_sat)
    V_sat_arr_dr = np.linspace(V_sat_dr-4,V_sat_dr,n_sat)
    V_r_arr_dr = np.linspace(V_r_dr,V_r_dr+ 4, n_r)
    J_r_arr_dr = rd_current(phi_c+V_r_arr_dr, T_e)

    V_data_dr = np.concatenate((V_sat_arr_dr,V_scl_dr,V_r_arr_dr))#
    J_data_dr0 = np.concatenate((J_sat_arr_dr,J_scl_dr,J_r_arr_dr))
    efficiencies_dr = np.zeros_like(V_data_dr)
    powers_dr = np.zeros_like(V_data_dr)
    V_a_dr = np.concatenate((phi_e-phi_c-V_sat_arr_dr,gamma_c_scl_dr*k_ev*T_e,np.zeros_like(V_r_arr_dr)))
    for i,V in enumerate(V_a_dr):
        J_data_dr[i] = J_data_dr0[i] * calculate_currfraction(T_e, V, drefprob)
        temp = calculate_efficiency(A_em, J_data_dr[i]*1e-4, J_coll, phi_e, T_e, rho_em,
                   rho_ew, rho_cw, phi_c, T_coll, np.log(J_sat/J_data_dr0[i])*k*T_e/e,
                   emiss_eff, T_env, applied_voltage=V_data_dr[i], single_cycle=1)
        efficiencies_dr[i] = temp['eta']
        powers_dr[i] = temp['P_load']
    ind1 = n_sat
    ind2 = np.argmax(powers_dr)
    ind3 = np.argmax(efficiencies_dr)
    data1_dr.append([T_ei, V_data_dr[ind1], J_data_dr[ind1], powers_dr[ind1], efficiencies_dr[ind1]])
    data2_dr.append([T_ei, V_data_dr[ind2], J_data_dr[ind2], powers_dr[ind2], efficiencies_dr[ind2]])
    data3_dr.append([T_ei, V_data_dr[ind3], J_data_dr[ind3], powers_dr[ind3], efficiencies_dr[ind3]])
    
    if T_ei % 100 == 0:
        fig, ax = plt.subplots()
        ax.plot(V_data, J_data*1e-4, label = 'current density')
#         ax.plot(V_data_dr, J_data_dr*1e-4,'--', label = 'current density, diffuse reflection')
#         ax.plot(V_data_sr, J_data_sr*1e-4,'.', label = 'current density, specular reflection')
        ax.plot(V_data, powers, label = 'power density')
#         ax.plot(V_data_dr, powers_dr,'--', label = 'power density, diffuse reflection')
#         ax.plot(V_data_sr, powers_sr,'.', label = 'power density, specular reflection')
        ax2 = ax.twinx()
        ax2.plot(V_data, efficiencies,'-k', label = 'efficiency')
#         ax2.plot(V_data_dr, efficiencies_dr,'--k', label = 'efficiency, diffuse reflection')
#         ax2.plot(V_data_sr, efficiencies_sr,'.k', label = 'efficiency, specular reflection')
        ax.set_xlabel('applied voltage V (V)')
        ax.set_ylabel('J (A/cm^2) and P (W/cm^2)')
        ax2.set_ylabel('efficiency')
        ax.legend(loc='upper left')
        ax2.legend(loc='upper right')
        plt.title(strT+'C')
        plt.savefig('J_P_efficiency_'+strT+'.png')

In [ ]:
eta1_no = np.zeros_like(T_es)
eta2_no = np.zeros_like(T_es)
eta3_no = np.zeros_like(T_es)
eta1_sr = np.zeros_like(T_es)
eta2_sr = np.zeros_like(T_es)
eta3_sr = np.zeros_like(T_es)
eta1_dr = np.zeros_like(T_es)
eta2_dr = np.zeros_like(T_es)
eta3_dr = np.zeros_like(T_es)
for i, T_e in enumerate(T_es):
    eta1_no[i] = data1_no[i][-1]
    eta1_sr[i] = data1_sr[i][-1]
    eta1_dr[i] = data1_dr[i][-1]
    eta2_no[i] = data2_no[i][-1]
    eta2_sr[i] = data2_sr[i][-1]
    eta2_dr[i] = data2_dr[i][-1]
    eta3_no[i] = data3_no[i][-1]
    eta3_sr[i] = data3_sr[i][-1]
    eta3_dr[i] = data3_dr[i][-1]
    
fig, ax = plt.subplots(figsize=(20,6))

eta = np.zeros_like(T_es)

ax.plot(T_es, eta1_no, label = 'peak current, no reflection')
ax.plot(T_es, eta1_sr, label = 'peak current, specular reflection')
ax.plot(T_es, eta1_dr, label = 'peak current, diffuse reflection')

ax.plot(T_es, eta2_no,'.', label = 'peak power, no reflection')
ax.plot(T_es, eta2_sr,'.', label = 'peak power, specular reflection')
ax.plot(T_es, eta2_dr,'.', label = 'peak power, diffuse reflection')

ax.plot(T_es, eta3_no,'o', label = 'peak efficiency, no reflection')
ax.plot(T_es, eta3_sr,'o', label = 'peak efficiency, specular reflection')
ax.plot(T_es, eta3_dr,'o', label = 'peak efficiency, diffuse reflection')

ax.legend(loc="upper left")

ax.set_xlim([1000, 5000])
ax.set_ylim([0.0, 1])

ax.set_title(dgap+' um')
ax.set_xlabel("T_em (C)")
ax.set_ylabel("efficiency, $\\eta$")

plt.savefig('efficiency'+dgap+'um.png')

In [ ]:
V1_no = np.zeros_like(T_es)
V2_no = np.zeros_like(T_es)
V3_no = np.zeros_like(T_es)
V1_sr = np.zeros_like(T_es)
V2_sr = np.zeros_like(T_es)
V3_sr = np.zeros_like(T_es)
V1_dr = np.zeros_like(T_es)
V2_dr = np.zeros_like(T_es)
V3_dr = np.zeros_like(T_es)
for i, T_e in enumerate(T_es):
    V1_no[i] = data1_no[i][1]
    V1_sr[i] = data1_sr[i][1]
    V1_dr[i] = data1_dr[i][1]
    V2_no[i] = data2_no[i][1]
    V2_sr[i] = data2_sr[i][1]
    V2_dr[i] = data2_dr[i][1]
    V3_no[i] = data3_no[i][1]
    V3_sr[i] = data3_sr[i][1]
    V3_dr[i] = data3_dr[i][1]
    
fig, ax = plt.subplots(figsize=(20,6))

eta = np.zeros_like(T_es)

ax.plot(T_es, V1_no, label = 'peak current, no reflection')
ax.plot(T_es, V1_sr, label = 'peak current, specular reflection')
ax.plot(T_es, V1_dr, label = 'peak current, diffuse reflection')

ax.plot(T_es, V2_no,'.', label = 'peak power, no reflection')
ax.plot(T_es, V2_sr,'.', label = 'peak power, specular reflection')
ax.plot(T_es, V2_dr,'.', label = 'peak power, diffuse reflection')

ax.plot(T_es, V3_no,'o', label = 'peak efficiency, no reflection')
ax.plot(T_es, V3_sr,'o', label = 'peak efficiency, specular reflection')
ax.plot(T_es, V3_dr,'o', label = 'peak efficiency, diffuse reflection')

ax.legend(loc="lower right")

ax.set_xlim([1000, 5000])
ax.set_ylim([0.0,8])

ax.set_xlabel("T_em (C)")
ax.set_ylabel("V_applied, $V$")
plt.savefig('applied_voltage_'+dgap+'um.png')